In [ ]:
import numpy as np
import gurobipy as gp

In [ ]:
M = 1000
N = 5
obj = np.array([M,132,217,164,58,132,M,290,201,79,217,290,M,113,303,164,201,113,M,196,58,79,303,196,M]+[0]*N)

In [ ]:
A = np.zeros((2*N + (N-1)**2 - (N-1),N**2 + N))
b = np.zeros(2*N + (N-1)**2 - (N-1))
direction = np.array(['']*(2*N + (N-1)**2 - (N-1)))

In [ ]:
ind_vec = np.array(range(N))
row = 0
for j in range(N):
    A[row,j*N + ind_vec] = 1
    b[row] = 1
    direction[row] = '='
    row+=1

In [ ]:
for i in range(N):
    A[row,ind_vec*N +i] = 1
    b[row] = 1
    direction[row] = '='
    row+=1

In [ ]:
for i in range(1,5):
    for j in range(1,5):
        if i != j:
            A[row,[N**2+i,N**2+j,j*N+i]] = [1,-1,N]
            b[row] = N-1
            direction[row] = '<'
            row+=1

In [ ]:
tspMod = gp.Model()
tspMod_x = tspMod.addMVar(N**2 + N,vtype=['B']*25+['C']*5) 
tspMod_con = tspMod.addMConstrs(A, tspMod_x, direction, b)
tspMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

tspMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
tspMod.optimize()

In [ ]:
tspMod_x.x

In [ ]:
tspMod.objval

In [ ]:
where = 0
miles_traveled = 0
print('Start in city 0.')
for city in range(N):
    where_new = np.where(tspMod_x.x[(where*N):((where+1)*N)])[0][0]
    dist = obj[where*N+where_new]
    miles_traveled += dist
    print('Then go to city '+str(where_new)+', traveling '+str(dist)+' miles along the way.')
    where = where_new
print('For a total of '+str(miles_traveled)+' miles.')

In [ ]:
# this path may not be unique, there are other paths that are optimal
# but they'll all have the same total miles traveled!
# for example 0 -> 2 -> 3 -> 1 -> 4 -> 0
# also takes 668 miles!

In [ ]:
seq = [0, 2, 3, 1, 4, 0]
miles_traveled2 = 0
for loc in range(5):
    where = seq[loc]
    where_new = seq[loc+1]
    dist = obj[where*N+where_new]
    miles_traveled2 += dist
print(miles_traveled2)

In [ ]:
# I have no idea how many other paths are optimal...